In [4]:
import pandas as pd
import os
import numpy as np
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS

In [5]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )
dir_data     = os.path.join(dir_inputs , "REMMdata"    )

BaseParcels        = os.path.join(dir_data  , r"parcels\Center01102019.gdb\BaseGP01102019"  )
Buildings          = os.path.join(dir_data  , r"BASE_run_285_2015_parcel_dev.csv"           )

ParcelsGDB = "parcels.gdb"

#name of new data features
Parcels     = os.path.join(dir_process, ParcelsGDB + "\Parcels"              )

#name of new data features
Parcels_shp = os.path.join(dir_process, "Parcels.shp"              )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)
        
#if processing geodatabase doesn't exist, create it
print("Checking if " + ParcelsGDB + " exists...")
if not arcpy.Exists(os.path.join(dir_process, ParcelsGDB)):
    print("Creating " + ParcelsGDB + "...")
    arcpy.management.CreateFileGDB(dir_process, ParcelsGDB)
else:
    print(ParcelsGDB + " exists...")


E:\GitHub\Parcel-Redevelopment-Potential
Checking if parcels.gdb exists...
parcels.gdb exists...


# Prep Parcels

In [6]:
#read in parcels
sdf_bPar = pd.DataFrame.spatial.from_featureclass(BaseParcels)

In [7]:
sdf_bPar

OBJECTID  parcel_id  parcel_id_REMM  basebldg Sliver    parcel_sqft  \
0              1   671124.0        671124.0       0.0   None   16203.648534   
1              2   588354.0        588354.0       0.0   None   49466.799191   
2              3   664267.0        664267.0       1.0   None   27944.879802   
3              4   632453.0        632453.0       1.0   None   15190.031848   
4              5   632450.0        632450.0       1.0   None   10092.376514   
...          ...        ...             ...       ...    ...            ...   
845215    845216   601655.0        886696.0       0.0   None  137565.519929   
845216    845217   601655.0        886697.0       0.0   None  213243.313865   
845217    845218   601655.0        886698.0       0.0   None  214262.868997   
845218    845219   601655.0        886699.0       0.0   None   55026.868822   
845219    845220        NaN             NaN       NaN   None            NaN   

       Split             x             y  parcel_acre  ...  type5  type6  \
0       None  1.572705e+06  7.294321e+06     0.371986  ...      t      f   
1       None  1.572481e+06  7.293816e+06     1.135606  ...      t      f   
2       None  1.571343e+06  7.291668e+06     0.641529  ...      f      f   
3       None  1.580054e+06  7.303541e+06     0.348717  ...      f      f   
4       None  1.580190e+06  7.303272e+06     0.231690  ...      f      f   
...      ...           ...           ...          ...  ...    ...    ...   
845215   1.0  1.573685e+06  7.220955e+06     3.158082  ...      f      f   
845216   1.0  1.573302e+06  7.220958e+06     4.895412  ...      f      f   
845217   1.0  1.572836e+06  7.220958e+06     4.918818  ...      f      f   
845218   1.0  1.572074e+06  7.220958e+06     1.263248  ...      f      f   
845219   NaN           NaN           NaN          NaN  ...   None   None   

        type7  type8  max_height  Agriculture  NoBuild  redev_friction  note  \
0           f      f       999.0          0.0      NaN             NaN  None   
1           f      f       999.0          0.0      NaN             NaN  None   
2           f      f       999.0          0.0      NaN             NaN  None   
3           f      f       999.0          0.0      NaN             NaN  None   
4           f      f       999.0          0.0      NaN             NaN  None   
...       ...    ...         ...          ...      ...             ...   ...   
845215      f      f       999.0          1.0      NaN            None  None   
845216      f      f       999.0          1.0      NaN            None  None   
845217      f      f       999.0          1.0      NaN            None  None   
845218      f      f       999.0          0.0      NaN            None  None   
845219   None   None         NaN          NaN      NaN            None  None   

                                                    SHAPE  
0       {"rings": [[[1572658.531162396, 7294412.922931...  
1       {"rings": [[[1572446.1962272525, 7294150.73555...  
2       {"rings": [[[1571352.9701225907, 7291780.92202...  
3       {"rings": [[[1579974.9191654772, 7303604.60588...  
4       {"rings": [[[1580242.158429563, 7303338.493389...  
...                                                   ...  
845215  {"rings": [[[1573850.763353318, 7220743.714214...  
845216  {"rings": [[[1573537.6948660016, 7220743.71421...  
845217  {"rings": [[[1573072.201039672, 7220743.714214...  
845218  {"rings": [[[1572137.6125155538, 7220743.71421...  
845219                                               None  

[845220 rows x 36 columns]

In [8]:
#set field for commercial, residential, or mixed

sdf_bPar['luType'] = ''

#undevelopable land (all building types are 'f')
sdf_bPar.loc[((sdf_bPar['type1'] == 'f') &
              (sdf_bPar['type2'] == 'f') &
              (sdf_bPar['type3'] == 'f') &
              (sdf_bPar['type4'] == 'f') &
              (sdf_bPar['type5'] == 'f') &
              (sdf_bPar['type6'] == 'f') &
              (sdf_bPar['type7'] == 'f') &
              (sdf_bPar['type8'] == 'f')), 'luType'] = 'Undevelopable'

sdf_bPar = sdf_bPar.drop(columns=['OBJECTID'      ,
                                  'city'          ,
                                  'x'             ,
                                  'y'             ,
                                  'county_name'   ,
                                  'distsml_id'    ,
                                  'distmed_id'    ,
                                  'distlrg_id'    ,
                                  'county_taz_id' ,
                                  'NoBuild'       ,
                                  'redev_friction',
                                  'Agriculture'   ,
                                  'Sliver'        ,
                                  'Split'         ,
                                  'utmxi'         ,
                                  'utmyi'         ,
                                  'note'          ,
                                  'zone_id'       ,
                                  'type1'         ,
                                  'type2'         ,
                                  'type3'         ,
                                  'type4'         ,
                                  'type5'         ,
                                  'type6'         ,
                                  'type7'         ,
                                  'type8'])

In [9]:
sdf_bPar

parcel_id  parcel_id_REMM  basebldg    parcel_sqft  parcel_acre  \
0        671124.0        671124.0       0.0   16203.648534     0.371986   
1        588354.0        588354.0       0.0   49466.799191     1.135606   
2        664267.0        664267.0       1.0   27944.879802     0.641529   
3        632453.0        632453.0       1.0   15190.031848     0.348717   
4        632450.0        632450.0       1.0   10092.376514     0.231690   
...           ...             ...       ...            ...          ...   
845215   601655.0        886696.0       0.0  137565.519929     3.158082   
845216   601655.0        886697.0       0.0  213243.313865     4.895412   
845217   601655.0        886698.0       0.0  214262.868997     4.918818   
845218   601655.0        886699.0       0.0   55026.868822     1.263248   
845219        NaN             NaN       NaN            NaN          NaN   

        county_id  max_dua  max_far  max_height  \
0             4.0      NaN     0.50       999.0   
1             4.0      NaN     0.50       999.0   
2             4.0      NaN     0.41       999.0   
3             4.0    4.000      NaN       999.0   
4             4.0    4.000      NaN       999.0   
...           ...      ...      ...         ...   
845215        4.0    0.025      NaN       999.0   
845216        4.0    0.025      NaN       999.0   
845217        4.0    0.025      NaN       999.0   
845218        4.0    0.025      NaN       999.0   
845219        NaN      NaN      NaN         NaN   

                                                    SHAPE luType  
0       {"rings": [[[1572658.531162396, 7294412.922931...         
1       {"rings": [[[1572446.1962272525, 7294150.73555...         
2       {"rings": [[[1571352.9701225907, 7291780.92202...         
3       {"rings": [[[1579974.9191654772, 7303604.60588...         
4       {"rings": [[[1580242.158429563, 7303338.493389...         
...                                                   ...    ...  
845215  {"rings": [[[1573850.763353318, 7220743.714214...         
845216  {"rings": [[[1573537.6948660016, 7220743.71421...         
845217  {"rings": [[[1573072.201039672, 7220743.714214...         
845218  {"rings": [[[1572137.6125155538, 7220743.71421...         
845219                                               None         

[845220 rows x 11 columns]

In [10]:
sdf_bPar_luTypeEmpty = sdf_bPar.loc[(sdf_bPar['luType'] == '')]
sdf_bPar_luTypeEmpty

parcel_id  parcel_id_REMM  basebldg    parcel_sqft  parcel_acre  \
0        671124.0        671124.0       0.0   16203.648534     0.371986   
1        588354.0        588354.0       0.0   49466.799191     1.135606   
2        664267.0        664267.0       1.0   27944.879802     0.641529   
3        632453.0        632453.0       1.0   15190.031848     0.348717   
4        632450.0        632450.0       1.0   10092.376514     0.231690   
...           ...             ...       ...            ...          ...   
845215   601655.0        886696.0       0.0  137565.519929     3.158082   
845216   601655.0        886697.0       0.0  213243.313865     4.895412   
845217   601655.0        886698.0       0.0  214262.868997     4.918818   
845218   601655.0        886699.0       0.0   55026.868822     1.263248   
845219        NaN             NaN       NaN            NaN          NaN   

        county_id  max_dua  max_far  max_height  \
0             4.0      NaN     0.50       999.0   
1             4.0      NaN     0.50       999.0   
2             4.0      NaN     0.41       999.0   
3             4.0    4.000      NaN       999.0   
4             4.0    4.000      NaN       999.0   
...           ...      ...      ...         ...   
845215        4.0    0.025      NaN       999.0   
845216        4.0    0.025      NaN       999.0   
845217        4.0    0.025      NaN       999.0   
845218        4.0    0.025      NaN       999.0   
845219        NaN      NaN      NaN         NaN   

                                                    SHAPE luType  
0       {"rings": [[[1572658.531162396, 7294412.922931...         
1       {"rings": [[[1572446.1962272525, 7294150.73555...         
2       {"rings": [[[1571352.9701225907, 7291780.92202...         
3       {"rings": [[[1579974.9191654772, 7303604.60588...         
4       {"rings": [[[1580242.158429563, 7303338.493389...         
...                                                   ...    ...  
845215  {"rings": [[[1573850.763353318, 7220743.714214...         
845216  {"rings": [[[1573537.6948660016, 7220743.71421...         
845217  {"rings": [[[1573072.201039672, 7220743.714214...         
845218  {"rings": [[[1572137.6125155538, 7220743.71421...         
845219                                               None         

[769845 rows x 11 columns]

In [11]:
sdf_bPar_lutypeTotals = sdf_bPar.groupby(['luType']).agg({"parcel_id": [np.size],"parcel_acre": [np.sum]})
sdf_bPar_lutypeTotals

parcel_id    parcel_acre
                   size            sum
luType                                
               769845.0  497497.352080
Undevelopable   75375.0  284066.102891

In [12]:
sdf_bParProcessed = sdf_bPar[['parcel_id_REMM','parcel_acre','luType','SHAPE']]

In [13]:
sdf_bParProcessed

parcel_id_REMM  parcel_acre luType  \
0             671124.0     0.371986          
1             588354.0     1.135606          
2             664267.0     0.641529          
3             632453.0     0.348717          
4             632450.0     0.231690          
...                ...          ...    ...   
845215        886696.0     3.158082          
845216        886697.0     4.895412          
845217        886698.0     4.918818          
845218        886699.0     1.263248          
845219             NaN          NaN          

                                                    SHAPE  
0       {"rings": [[[1572658.531162396, 7294412.922931...  
1       {"rings": [[[1572446.1962272525, 7294150.73555...  
2       {"rings": [[[1571352.9701225907, 7291780.92202...  
3       {"rings": [[[1579974.9191654772, 7303604.60588...  
4       {"rings": [[[1580242.158429563, 7303338.493389...  
...                                                   ...  
845215  {"rings": [[[1573850.763353318, 7220743.714214...  
845216  {"rings": [[[1573537.6948660016, 7220743.71421...  
845217  {"rings": [[[1573072.201039672, 7220743.714214...  
845218  {"rings": [[[1572137.6125155538, 7220743.71421...  
845219                                               None  

[845220 rows x 4 columns]

In [14]:
df_parcelwithbldg = pd.read_csv(Buildings)

In [15]:
display(Buildings)

'E:\\GitHub\\Parcel-Redevelopment-Potential\\input\\REMMdata\\BASE_run_285_2015_parcel_dev.csv'

In [16]:
pd.set_option('display.max_columns', None)
df_parcelwithbldg

parcel_id  parcel_id_REMM  county_id  zone_id  parcel_acres  \
0          741871          741871         49     2111      0.197968   
1          579822          579822         49     2873     37.717834   
2          579853          579853         49     2117      6.791235   
3          640185          640185         49     2073      1.372797   
4          682698          682698         49     2119      0.163751   
...           ...             ...        ...      ...           ...   
841679     886695          886695         49     2666      4.531522   
841680     886696          886696         49     2666      3.158082   
841681     886697          886697         49     2666      4.895412   
841682     886698          886698         49     2666      4.918818   
841683     886699          886699         49     2666      1.263248   

        land_value  max_far  max_dua  has_buildings  was_developed  \
0         35700.00      NaN    6.000              1              0   
1         56614.32      NaN      NaN              0              0   
2         81500.00      NaN    6.000              0              0   
3        112100.00      NaN    2.500              1              0   
4         43000.00      NaN    0.200              1              0   
...            ...      ...      ...            ...            ...   
841679    48006.00      NaN    0.025              0              0   
841680    33456.00      NaN    0.025              0              0   
841681    51862.00      NaN    0.025              0              0   
841682    52109.00      NaN    0.025              0              0   
841683    13383.00      NaN    0.025              0              0   

        was_redeveloped  developable  residential_units  job_spaces  \
0                     0            1                  1           0   
1                     0            0                  0           0   
2                     0            1                  0           0   
3                     0            1                  1           0   
4                     0            1                  1           0   
...                 ...          ...                ...         ...   
841679                0            1                  0           0   
841680                0            1                  0           0   
841681                0            1                  0           0   
841682                0            1                  0           0   
841683                0            1                  0           0   

        building_sqft  non_residential_sqft  residential_sqft  \
0              2434.0                   0.0            2434.0   
1                 0.0                   NaN               0.0   
2                 0.0                   NaN               0.0   
3              5499.0                   0.0            5499.0   
4              3046.0                   0.0            3046.0   
...               ...                   ...               ...   
841679            0.0                   NaN               0.0   
841680            0.0                   NaN               0.0   
841681            0.0                   NaN               0.0   
841682            0.0                   NaN               0.0   
841683            0.0                   NaN               0.0   

        unit_price_non_residential  res_price_per_sqft  is_sf  is_mf  \
0                              0.0           74.185722    1.0    0.0   
1                              NaN                 NaN    NaN    NaN   
2                              NaN                 NaN    NaN    NaN   
3                              0.0           45.462811    1.0    0.0   
4                              0.0           61.906857    1.0    0.0   
...                            ...                 ...    ...    ...   
841679                         NaN                 NaN    NaN    NaN   
841680                         NaN                 NaN    NaN    NaN   
841681                         N

In [17]:
df_bldg = df_parcelwithbldg[df_parcelwithbldg['has_buildings']==1].copy()

df_bldg['bldgtype'] = ''

#residential building types
df_bldg.loc[(df_bldg['is_sf'] != 0) | (df_bldg['is_mf'] != 0), 'bldgtype'] = 'Residential'

#commercial building types, if mixed use (res and com) then use commercial 
df_bldg.loc[((df_bldg['is_industrial'] != 0) |
             (df_bldg['is_retail'    ] != 0) |
             (df_bldg['is_office'    ] != 0) |
             (df_bldg['is_govt'      ] != 0) |
             (df_bldg['is_mixeduse'  ] != 0) |
             (df_bldg['is_other'     ] != 0)), 'bldgtype'] = 'Commercial'

df_bldg

parcel_id  parcel_id_REMM  county_id  zone_id  parcel_acres  \
0          741871          741871         49     2111      0.197968   
3          640185          640185         49     2073      1.372797   
4          682698          682698         49     2119      0.163751   
6          693553          693553         49     2147      0.487942   
7          658088          658088         49     2143      0.337967   
...           ...             ...        ...      ...           ...   
841528     601446          601446         49     2651      2.902050   
841575     609702          609702         49     2731      0.515341   
841587     603550          603550         49     2687      4.903986   
841621     581782          581782         49     2069     35.460532   
841663     601655          601655         49     2666      3.376715   

        land_value  max_far  max_dua  has_buildings  was_developed  \
0          35700.0      NaN    6.000              1              0   
3         112100.0      NaN    2.500              1              0   
4          43000.0      NaN    0.200              1              0   
6          87700.0      NaN    3.000              1              0   
7          76200.0      NaN      NaN              1              0   
...            ...      ...      ...            ...            ...   
841528     14492.0      NaN      NaN              1              0   
841575      7146.0      NaN    6.000              1              0   
841587     60519.0      NaN    0.200              1              0   
841621   1261202.0      NaN    2.500              1              0   
841663     35773.0      NaN    0.025              1              0   

        was_redeveloped  developable  residential_units  job_spaces  \
0                     0            1                  1           0   
3                     0            1                  1           0   
4                     0            1                  1           0   
6                     0            1                  1           0   
7                     0            0                  1           0   
...                 ...          ...                ...         ...   
841528                0            0                  1           0   
841575                0            1                  1           0   
841587                0            1                  1           0   
841621                0            1                  1           0   
841663                0            1                  1           0   

        building_sqft  non_residential_sqft  residential_sqft  \
0              2434.0                   0.0            2434.0   
3              5499.0                   0.0            5499.0   
4              3046.0                   0.0            3046.0   
6              3268.0                   0.0            3268.0   
7              4408.0                   0.0            4408.0   
...               ...                   ...               ...   
841528         2935.0                   0.0            2935.0   
841575         2958.0                   0.0            2958.0   
841587         1296.0                   0.0            1296.0   
841621         1414.0                   0.0            1414.0   
841663          678.0                   0.0             678.0   

        unit_price_non_residential  res_price_per_sqft  is_sf  is_mf  \
0                              0.0           74.185722    1.0    0.0   
3                              0.0           45.462811    1.0    0.0   
4                              0.0           61.906857    1.0    0.0   
6                              0.0           56.303550    1.0    0.0   
7                              0.0            0.000000    1.0    0.0   
...                            ...                 ...    ...    ...   
841528                         0.0            0.000000    1.0    0.0   
841575                         0.0           36.308316    1.0    0.0   
841587                         0

In [18]:
df_bldg[df_bldg['parcel_id']==77033]

parcel_id  parcel_id_REMM  county_id  zone_id  parcel_acres  \
443785      77033           77033         35      969      6.136467   

        land_value  max_far  max_dua  has_buildings  was_developed  \
443785  4282341.82     9.37    125.0              1              0   

        was_redeveloped  developable  residential_units  job_spaces  \
443785                0            1                144           0   

        building_sqft  non_residential_sqft  residential_sqft  \
443785       161779.0                   0.0          161779.0   

        unit_price_non_residential  res_price_per_sqft  is_sf  is_mf  \
443785                         0.0                 0.0    0.0    1.0   

        is_industrial  is_retail  is_office  is_govt  is_mixeduse  is_other  \
443785            0.0        0.0        0.0      0.0          0.0       0.0   

        year_built  building_count  non_res_value  res_value  total_value  \
443785      2011.0             144            0.0        0.0   4282341.82   

           bldgtype  
443785  Residential

In [19]:
#"bldgtype":[np.min] --> 'Commercial' < 'Residential', pick commercial if a mix of types

df_bldg_summary = df_bldg.groupby(['parcel_id'], as_index=False).agg({"bldgtype":[np.min], "year_built":[np.max], "residential_units":[np.sum], "job_spaces":[np.sum], "residential_sqft":[np.sum], "non_residential_sqft":[np.sum], "res_price_per_sqft":[np.mean], "unit_price_non_residential":[np.mean]})
df_bldg_summary.columns = df_bldg_summary.columns.droplevel(1)

df_bldg_summary['value_res'] = df_bldg_summary['residential_sqft'] * df_bldg_summary['res_price_per_sqft']
df_bldg_summary['value_com'] = df_bldg_summary['non_residential_sqft'] * df_bldg_summary['unit_price_non_residential']

df_bldg_summary.columns = ('parcel_id','bldgtype','yearbuilt','resunits','job_spaces','sf_res','sf_com','sfvalue_res','sfvalue_com','value_res','value_com')

df_bldg_summary.to_csv(os.path.join(dir_process, "bldgsummary.csv"))
df_bldg_summary

parcel_id     bldgtype  yearbuilt  resunits  job_spaces  sf_res  \
0              44   Commercial     1952.0         0          39     0.0   
1             152  Residential     1951.0         1           0  2198.0   
2             215   Commercial     1979.0         0         260     0.0   
3             278   Commercial     1914.0         0          12     0.0   
4             308   Commercial     1948.0         0          14     0.0   
...           ...          ...        ...       ...         ...     ...   
615793     871288  Residential     2007.0         1           0  3269.0   
615794     872441  Residential     1932.0         1           0  1680.0   
615795     875208  Residential     2014.0         1           0  2971.0   
615796     882712   Commercial     1996.0         0           1     0.0   
615797     883250   Commercial     1958.0         0           4     0.0   

         sf_com  sfvalue_res  sfvalue_com  value_res     value_com  
0       14586.0          0.0     0.000000        0.0      0.000000  
1           0.0          0.0     0.000000        0.0      0.000000  
2       97240.0          0.0     0.000000        0.0      0.000000  
3        4488.0          0.0     0.000000        0.0      0.000000  
4       13650.0          0.0     0.000000        0.0      0.000000  
...         ...          ...          ...        ...           ...  
615793      0.0          0.0     0.000000        0.0      0.000000  
615794      0.0          0.0     0.000000        0.0      0.000000  
615795      0.0          0.0     0.000000        0.0      0.000000  
615796    350.0          0.0    30.428571        0.0  10650.000001  
615797   3900.0          0.0     0.000000        0.0      0.000000  

[615798 rows x 11 columns]

In [20]:
sdf_bPar_bldg = pd.DataFrame.merge(sdf_bPar,df_bldg_summary,on='parcel_id',how='left')
sdf_bPar_bldg

parcel_id  parcel_id_REMM  basebldg    parcel_sqft  parcel_acre  \
0        671124.0        671124.0       0.0   16203.648534     0.371986   
1        588354.0        588354.0       0.0   49466.799191     1.135606   
2        664267.0        664267.0       1.0   27944.879802     0.641529   
3        632453.0        632453.0       1.0   15190.031848     0.348717   
4        632450.0        632450.0       1.0   10092.376514     0.231690   
...           ...             ...       ...            ...          ...   
845215   601655.0        886696.0       0.0  137565.519929     3.158082   
845216   601655.0        886697.0       0.0  213243.313865     4.895412   
845217   601655.0        886698.0       0.0  214262.868997     4.918818   
845218   601655.0        886699.0       0.0   55026.868822     1.263248   
845219        NaN             NaN       NaN            NaN          NaN   

        county_id  max_dua  max_far  max_height  \
0             4.0      NaN     0.50       999.0   
1             4.0      NaN     0.50       999.0   
2             4.0      NaN     0.41       999.0   
3             4.0    4.000      NaN       999.0   
4             4.0    4.000      NaN       999.0   
...           ...      ...      ...         ...   
845215        4.0    0.025      NaN       999.0   
845216        4.0    0.025      NaN       999.0   
845217        4.0    0.025      NaN       999.0   
845218        4.0    0.025      NaN       999.0   
845219        NaN      NaN      NaN         NaN   

                                                    SHAPE luType     bldgtype  \
0       {'rings': [[[1572658.531162396, 7294412.922931...                 NaN   
1       {'rings': [[[1572446.1962272525, 7294150.73555...                 NaN   
2       {'rings': [[[1571352.9701225907, 7291780.92202...          Commercial   
3       {'rings': [[[1579974.9191654772, 7303604.60588...         Residential   
4       {'rings': [[[1580242.158429563, 7303338.493389...         Residential   
...                                                   ...    ...          ...   
845215  {'rings': [[[1573850.763353318, 7220743.714214...         Residential   
845216  {'rings': [[[1573537.6948660016, 7220743.71421...         Residential   
845217  {'rings': [[[1573072.201039672, 7220743.714214...         Residential   
845218  {'rings': [[[1572137.6125155538, 7220743.71421...         Residential   
845219                                               None                 NaN   

        yearbuilt  resunits  job_spaces  sf_res  sf_com  sfvalue_res  \
0             NaN       NaN         NaN     NaN     NaN          NaN   
1             NaN       NaN         NaN     NaN     NaN          NaN   
2          2010.0       0.0         4.0     0.0  3900.0     0.000000   
3          1998.0       1.0         0.0  2194.0     0.0    78.133578   
4          1997.0       1.0         0.0  1698.0     0.0    83.941064   
...           ...       ...         ...     ...     ...          ...   
845215     1892.0       1.0         0.0   678.0     0.0    40.560472   
845216     1892.0       1.0         0.0   678.0     0.0    40.560472   
845217     1892.0       1.0         0.0   678.0     0.0    40.560472   
845218     1892.0       1.0         0.0   678.0     0.0    40.560472   
845219        NaN       NaN         NaN     NaN     NaN          NaN   

        sfvalue_com      value_res  value_com  
0               NaN            NaN        NaN  
1               NaN            NaN        NaN  
2               0.0       0.000000        0.0  
3               0.0  171425.070988        0.0  
4               0.0  142531.926978        0.0  
...             ...            ...        ...  
845215          0.0   27500.000002        0.0  
845216          0.0   27500.000002        0.0  
845217          0.0   27500.000002        0.0  
845218          0.0   27500.000002        0.0  
845219          NaN            NaN        NaN  

[845220 rows x 21 columns]

In [21]:
#sdf_bPar_cParMax_bldg.sum()

In [22]:
print ("Exporting intermediate Parcel feature class...")
sdf_bPar_bldg.spatial.to_featureclass(location=Parcels)

print ("Calculating parcel acreage...")

# Execute DeleteField
arcpy.DeleteField_management(Parcels, ['Acres'])

arcpy.AddField_management(Parcels, "ParcelAcres" , "DOUBLE")
arcpy.CalculateField_management(Parcels,
                                "ParcelAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Done")

'E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\parcels.gdb\\Parcels'

In [23]:
#clear memory
del sdf_bPar_bldg
del sdf_bPar
del df_bldg
del df_bldg_summary